# House Prices - Advanced Regression Techniques

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
y_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

## Data fields

**SalePrice** - the property's sale price in dollars. This is the target variable that you're trying to predict.

**MSSubClass**: The building class

**MSZoning**: The general zoning classification

**LotFrontage**: Linear feet of street connected to property

**LotArea**: Lot size in square feet

**Street**: Type of road access

**Alley**: Type of alley access

**LotShape**: General shape of property

**LandContour**: Flatness of the property

**Utilities**: Type of utilities available

**LotConfig**: Lot configuration

**LandSlope**: Slope of property

**Neighborhood**: Physical locations within Ames city limits

**Condition1**: Proximity to main road or railroad

**Condition2**: Proximity to main road or railroad (if a second is present)

**BldgType**: Type of dwelling

**HouseStyle**: Style of dwelling

**OverallQual**: Overall material and finish quality

**OverallCond**: Overall condition rating

**YearBuilt**: Original construction date

**YearRemodAdd**: Remodel date

**RoofStyle**: Type of roof

**RoofMatl**: Roof material

**Exterior1st**: Exterior covering on house

**Exterior2nd**: Exterior covering on house (if more than one material)

**MasVnrType**: Masonry veneer type

**MasVnrArea**: Masonry veneer area in square feet

**ExterQual**: Exterior material quality

**ExterCond**: Present condition of the material on the exterior

**Foundation**: Type of foundation

**BsmtQual**: Height of the basement

**BsmtCond**: General condition of the basement

**BsmtExposure**: Walkout or garden level basement walls

**BsmtFinType1**: Quality of basement finished area

**BsmtFinSF1**: Type 1 finished square feet

**BsmtFinType2**: Quality of second finished area (if present)

**BsmtFinSF2**: Type 2 finished square feet

**BsmtUnfSF**: Unfinished square feet of basement area

**TotalBsmtSF**: Total square feet of basement area

**Heating**: Type of heating

**HeatingQC**: Heating quality and condition

**CentralAir**: Central air conditioning

**Electrical**: Electrical system

**1stFlrSF**: First Floor square feet

**2ndFlrSF**: Second floor square feet

**LowQualFinSF**: Low quality finished square feet (all floors)

**GrLivArea**: Above grade (ground) living area square feet

**BsmtFullBath**: Basement full bathrooms

**BsmtHalfBath**: Basement half bathrooms

**FullBath**: Full bathrooms above grade

**HalfBath**: Half baths above grade

**Bedroom**: Number of bedrooms above basement level

**Kitchen**: Number of kitchens

**KitchenQual**: Kitchen quality

**TotRmsAbvGrd**: Total rooms above grade (does not include bathrooms)

**Functional**: Home functionality rating

**Fireplaces**: Number of fireplaces

**FireplaceQu**: Fireplace quality

**GarageType**: Garage location

**GarageYrBlt**: Year garage was built

**GarageFinish**: Interior finish of the garage

**GarageCars**: Size of garage in car capacity

**GarageArea**: Size of garage in square feet

**GarageQual**: Garage quality

**GarageCond**: Garage condition

**PavedDrive**: Paved driveway

**WoodDeckSF**: Wood deck area in square feet

**OpenPorchSF**: Open porch area in square feet

**EnclosedPorch**: Enclosed porch area in square feet

**3SsnPorch**: Three season porch area in square feet

**ScreenPorch**: Screen porch area in square feet

**PoolArea**: Pool area in square feet

**PoolQC**: Pool quality

**Fence**: Fence quality

**MiscFeature**: Miscellaneous feature not covered in other categories

**MiscVal**: $Value of miscellaneous feature

**MoSold**: Month Sold

**YrSold**: Year Sold

**SaleType**: Type of sale

**SaleCondition**: Condition of sale

# Data Analysys

In [ ]:
train.head()

In [ ]:
train.info()


we have dtypes: **float-3, int-34, object-43** 

We also see that there are columns with missing values, some of them are almost completely empty, let's study them first

## Check columns with the most missing values

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(train.isnull());

**Alley:** Type of alley access

**FireplaceQu:** Fireplace quality

**PoolQC:** Pool quality

**Fence:** Fence quality

**MiscFeature:** Miscellaneous feature not covered in other categories

In [ ]:
print(
    'Alley:',train['Alley'].unique(), round(train['Alley'].count()/train.shape[0],3),'%',
    '\nFireplaceQu:',train['FireplaceQu'].unique(), round(train['FireplaceQu'].count()/train.shape[0],3),'%',
    '\nPoolQC:',train['PoolQC'].unique(), round(train['PoolQC'].count()/train.shape[0],3),'%',
    '\nFence:',train['Fence'].unique(), round(train['Fence'].count()/train.shape[0],3),'%',
    '\nMiscFeature:',train['MiscFeature'].unique(), round(train['MiscFeature'].count()/train.shape[0],3),'%',
    )


--------------------------
If we analyze the variables in more detail, we can assume that

that NaN in this case is not a missing, but an identifier that there is no presence of a certain object in the house.

For example, **Alley** NaN may mean that there is no access through the alley to the house

**FireplaceQu** NaN means that there is no fireplace in the house at all

**PoolQC** NaN means no pool

**Fence** NaN means no fence

**MiscFeature** NaN means the absence of various features

--------------------

But this assumption does not have to be true or it can be half true, in which case we will mark the real NaNs as an absence target, which is also not a good solution.


I'll still assume that NaN is indeed the absence factor and re-encode to class **'No'**

In [ ]:
train['Alley'].fillna('No', inplace=True)
train['FireplaceQu'].fillna('No', inplace=True)
train['PoolQC'].fillna('No', inplace=True)
train['Fence'].fillna('No', inplace=True)
train['MiscFeature'].fillna('No', inplace=True)

test['Alley'].fillna('No', inplace=True)
test['FireplaceQu'].fillna('No', inplace=True)
test['PoolQC'].fillna('No', inplace=True)
test['Fence'].fillna('No', inplace=True)
test['MiscFeature'].fillna('No', inplace=True)

In [ ]:
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

We will also remove the **Id** column, which does not carry any meaning.

We'll deal with the rest of the missings values later.

Now let's look at the target variable and its correlation with the independent variables

# Target Variable Analysis

In [ ]:
train['SalePrice'].describe()

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(train['SalePrice'],kde=True);

We have several outliers in the target variable, we can use the **log transformation** to normalize, but first let's look at **probplot**

In [ ]:
fig = plt.figure(figsize = (15,10))
fig.add_subplot(1,2,1)
res = stats.probplot(train['SalePrice'], plot=plt)
fig.add_subplot(1,2,2)
res = stats.probplot(np.log(train['SalePrice']), plot=plt)

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(np.log(train['SalePrice']),kde=True);

The target variable is now **normally distributed**

In [ ]:
train['SalePrice'] = np.log(train['SalePrice'])

Let's look at the **correlation matrix**

I won't use annot=True in the heatmap settings as the numbers will overlap and the visualization won't be readable, instead I'll just set the threshold by which we look at the correlation, for example 0.5.

In order not to miss the negative correlation, we apply abs to matrix

Next, I will numerically derive the most important relationships with the target variable and their true values.

In [ ]:
main_corr_map = abs(train.corr())>0.5

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(main_corr_map, linewidths=3, linecolor='black');

We have selected **11** variables that are most correlated with the target, let's study them

In [ ]:
main_corr = train.corr()['SalePrice'][abs(train.corr())['SalePrice']>0.5].head(11).sort_values(ascending=False)
main_corr

In [ ]:
main_corr_name = list(main_corr.index)

In [ ]:
fig, ax =plt.subplots(3,4, figsize=(17,10))
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[0], ax=ax[0,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[1], ax=ax[0,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[2], ax=ax[0,2])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[3], ax=ax[0,3])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[4], ax=ax[1,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[5], ax=ax[1,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[6], ax=ax[1,2])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[7], ax=ax[1,3])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[8], ax=ax[2,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[9], ax=ax[2,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[10], ax=ax[2,2]);

Indeed, there is an almost linear positive relationship with many variables

On several scatterplots, such as **TotalBsmtSF**, **GrLivArea**, **TotRmsAbvGrd**, **GarageArea**, outliers are visible that can greatly affect the correlation with the target variable, you need to remove them

In [ ]:
train.drop(train[train['TotalBsmtSF']>6000].index, inplace=True)
train.drop(train[train['GrLivArea']>4500].index, inplace=True)
train.drop(train[train['TotRmsAbvGrd']>12.5].index, inplace=True)
train.drop(train[train['GarageArea']>1220].index, inplace=True)

Now let's look again at how the correlation with the target variable has changed

In [ ]:
after_main_corr = train.corr()['SalePrice'][abs(train.corr())['SalePrice']>0.5].head(11).sort_values(ascending=False)
after_main_corr

In [ ]:
after_main_corr - main_corr

The correlation has not increased much, but in any case, outliers are not desirable and they need to be removed

Take another look at the scatterplots without outliers

In [ ]:
fig, ax =plt.subplots(3,4, figsize=(20,10))
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[0], ax=ax[0,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[1], ax=ax[0,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[2], ax=ax[0,2])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[3], ax=ax[0,3])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[4], ax=ax[1,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[5], ax=ax[1,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[6], ax=ax[1,2])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[7], ax=ax[1,3])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[8], ax=ax[2,0])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[9], ax=ax[2,1])
sns.scatterplot(data=train, y='SalePrice', x=main_corr_name[10], ax=ax[2,2]);

Looks better than it was

As we observed in the correlation matrix, many variables correlate not only with the target variable, but also with each other, such relationships may well be multicollinearity, which will damage our future model.

# Analysis of nominative variables

Let's have a look at the correlation matrix again, but increase the correlation selection threshold to 0.65 to catch linear relationships between variables

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(abs(train.drop(['SalePrice'], axis=1).corr())>0.6, linewidths=3, linecolor='black');

Let's analyze each relationship separately, in the future I use PCA to eliminate multicollinearity, but for analysis it is worth looking at them in more detail

In [ ]:
train.select_dtypes(exclude=['object']).columns

### 1stFlrSF :TotalBsmtSF

**1stFlrSF**: First Floor square feet

**TotalBsmtSF**: Total square feet of basement area

In [ ]:
sns.heatmap(train[['TotalBsmtSF','1stFlrSF']].corr(), annot=True);

It is logical to assume that the relationship between these variables is **natural**, because the size of the basement will obviously depend on the size of the 1st floor of the house, but let's make sure of this and see the plots

-------------

First, let's build a boxplot for each variable

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.histplot(data= train, x='1stFlrSF', ax=ax[0])
sns.histplot(data= train, x='TotalBsmtSF', ax=ax[1]);

We see that the distribution of these variables is not normal due to values more than 2500, as well as due to the high number of values where there is no basement in the houses

However, you can apply a **log transformation to 1stFlrSF**, thereby possibly improving the linear relationship with the target variable and trying to get rid of it between these two features manually, but before that let's look at the scatterplot

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(data=train, x='1stFlrSF', y='TotalBsmtSF');

We see a direct linear relationship between these features, not counting the values of 0 and a few outliers

----------------------------

To **generate new features**, you can add a new categorical feature responsible for the presence of a basement in the house, becouse see a lot of values equal to 0

In [ ]:
train['CheckBsmt'] = train['TotalBsmtSF'].apply(lambda x: 0 if x==0  else 1)
test['CheckBsmt'] = test['TotalBsmtSF'].apply(lambda x: 0 if x==0  else 1)

And also apply the log transformation to **1stFlrSF**

In [ ]:
train['1stFlrSF'] = np.log(train['1stFlrSF'] )

### 2ndFlrSF : GrLivArea, HalfBath, TotRmsAbvGrd

**2ndFlrSF**: Second floor square feet

**GrLivArea**: Above grade (ground) living area square feet

**HalfBath**: Half baths above grade

**TotRmsAbvGrd**: Total rooms above grade (does not include bathrooms)


In [ ]:
sns.heatmap(train[['2ndFlrSF','GrLivArea','HalfBath', 'TotRmsAbvGrd']].corr(), annot=True);

This variable correlates with three at once, it may be worth removing it, but first, let's look at the relationships

In [ ]:
fig, ax =plt.subplots(1,4, figsize=(15,5))
sns.histplot(data= train, x='2ndFlrSF', ax=ax[0])
sns.histplot(data= train, x='GrLivArea', ax=ax[1]);
sns.histplot(data= train, x='TotRmsAbvGrd', ax=ax[2]);
sns.histplot(data= train, x='HalfBath', ax=ax[3]);

A variable that correlates with all the others in the same way as **TotalBsmtSF**, which we analyzed earlier, is responsible for the area. In this case, for the area of 2 floors, **we can generate another new feature** that is responsible for the presence of 2 floors in the house

**GrLivArea**, you can see that the feature is not normally distributed, but you can **apply a log transformation**

**TotRmsAbvGrd** seems to be quite an important variable for the number of rooms in the entire house, not including bathrooms.

While the **correlation between 2ndFlrSF and HalfBath is not clear**, it is possible that the presence of 2 floors directly affects the class of the bath in the house

----------------------

It makes sense to look at the scatterplot only for **2ndFlrSF** and **GrLivArea**, the rest of the signs are categorical and the scatterplot will not bring much information

Let's also look at **GrLivArea** and **TotRmsAbvGrd**, as there is a very strong correlation

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.scatterplot(data=train, x='2ndFlrSF', y='GrLivArea', ax=ax[0]);
sns.scatterplot(data=train, x='GrLivArea', y='TotRmsAbvGrd', ax=ax[1]);

Scatteplot shows a direct linear relationship, ignoring 0 values

As mentioned above, let's create a new feature that is responsible for the presence of 2 floors in the house

In [ ]:
train['Check2ndFlr'] = train['2ndFlrSF'].apply(lambda x: 0 if x==0  else 1)
test['Check2ndFlr'] = test['2ndFlrSF'].apply(lambda x: 0 if x==0  else 1)
train['GrLivArea'] = np.log(train['GrLivArea'] )

### BsmtFullBath : BsmtFinSF1
### TotRmsAbvGrd : BedroomAbvGr
### GarageYrBlt: YearBuilt, YearRemodAdd

**BsmtFullBath**: Basement full bathrooms

**BsmtFinSF1**: Type 1 finished square feet

-----------

**TotRmsAbvGrd**: Total rooms above grade (does not include bathrooms)

**Bedroom**: Number of bedrooms above basement level


-------------

**GarageYrBlt**: Year garage was built

**YearBuilt**: Original construction date

**YearRemodAdd**: Remodel date




In [ ]:
sns.heatmap(train[['BsmtFullBath','BsmtFinSF1','BedroomAbvGr', 'GarageYrBlt','YearBuilt','YearRemodAdd']].corr(), annot=True);

Each of these variables is quite logically correlated with others, let's look at them

TotRmsAbvGrd we analyzed earlier

In [ ]:
fig, ax =plt.subplots(2,3, figsize=(15,7))
sns.histplot(data= train, x='BsmtFullBath', ax=ax[0,0])
sns.histplot(data= train, x='BsmtFinSF1', ax=ax[0,1]);
sns.histplot(data= train, x='BedroomAbvGr', ax=ax[0,2]);
sns.histplot(data= train, x='GarageYrBlt', ax=ax[1,0]);
sns.histplot(data= train, x='YearBuilt', ax=ax[1,1])
sns.histplot(data= train, x='YearRemodAdd', ax=ax[1,2]);


For BsmtFinSF1, we will do the same as with **TotalBsmtSF** and **2ndFlrSF**, we **will create the CheckBsmtFin feature**

YearBuilt seems to be the most important variable of the 3, but **we can create a new feature** that counts the difference between the time the house was built and when it was repaired.

In [ ]:
train['CheckBsmtFin'] = train['BsmtFinSF1'].apply(lambda x: 0 if x==0  else 1)
test['CheckBsmtFin'] = test['BsmtFinSF1'].apply(lambda x: 0 if x==0  else 1)

In [ ]:
train['YearDifference'] = train['YearRemodAdd'] - train['YearBuilt']
test['YearDifference'] = test['YearRemodAdd'] - test['YearBuilt']

In [ ]:
train['YearDifference'].unique()

### GarageCars : GarageArea

**GarageCars**: Size of garage in car capacity

**GarageArea**: Size of garage in square feet

Both variables almost repeat each other, the logic of using two such variables at once is not entirely clear, let's delete one of them

We will also **create a new feature** responsible for the presence of a garage in the house

In [ ]:
train['CheckGarage'] = train['GarageArea'].apply(lambda x: 0 if x==0 else 1)
test['CheckGarage'] = test['GarageArea'].apply(lambda x: 0 if x==0 else 1)

train.drop(['GarageCars'],axis=1, inplace=True)
test.drop(['GarageCars'],axis=1, inplace=True)

Great, we have looked at all the significant relationships between the nominal variables, it may be worth looking into the remaining nominative features in more detail and trying to generate even more new features, but we will do this another time

However, I **will add one feature that immediately catches your eye after studying and generating the 3 previous ones** - the total area of the house, add it and deal with categorical features

In [ ]:
train['TotalSF'] = train['TotalBsmtSF'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']

Now let's get into the analysis of categorical features

# Analysis of categorical variables

In [ ]:
train_object = train.select_dtypes(include=['object'])

In [ ]:
train_object.head()

In [ ]:
train_object.describe().T

We can see feture in which one of the classes of the feture strongly predominates

For example **Utilities** has a ratio of 1453:1

Such feature will not help us much, so we will remove those in which more than 90% is dominated by one class

In [ ]:
train_object.shape[1]

In [ ]:
check_object = []
for col in train_object.columns:
    if train_object[col].value_counts().values[0]>train_object.shape[0]*0.9:
        check_object.append(col)
train.drop(check_object, axis=1, inplace=True)
test.drop(check_object, axis=1, inplace=True)

In [ ]:
train_object = train.select_dtypes(include=['object'])
train_object.shape[1]

In [ ]:
train_object.describe().T

Great, but we still have a lot of categorical variables

Let's look at the boxplot for each and analyze the effect on the target variable

To make the visualization process more convenient, we divide the object variables into 5 equal parts

**Neighborhood**, **Exterior1st**, **Exterior2nd**, **Condition1**, **SaleType**, **HouseStyle** visualize separately as they have many classes

### Neighborhood, Exterior1st, Exterior2nd, Condition1, SaleType, HouseStyle

#### Exterior1st, Exterior2nd

**Exterior1st**: Exterior covering on house

**Exterior2nd**: Exterior covering on house (if more than one material)

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.countplot(data=train, x='Exterior1st', palette='coolwarm', ax=ax[0])
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, horizontalalignment='right')
sns.countplot(data=train, x='Exterior2nd',  palette='coolwarm', ax=ax[1])
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, horizontalalignment='right');

Each of these classes has one of the most popular **Vinyls** class and about 3-4 equally distributed classes which are half  smaller than the most popular class, the other classes are rare

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.boxplot(data= train, y='Exterior1st', x='SalePrice', palette='coolwarm', ax=ax[0])
sns.boxplot(data= train, y='Exterior2nd', x='SalePrice', palette='coolwarm', ax=ax[1]);

It can be seen that the type of class affects the Sale Price, for example, the **VinylSd** box expresses the high cost of the house more strongly than **MetalSD** or **Wd Shng**, even though they fall into the VinylSd whiskers zone

#### Neighborhood

**Neighborhood**: Physical locations within Ames city limits

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=train, x='Neighborhood', palette='rainbow')
plt.xticks(rotation=45);

The **NAmes** class is more common than the others, but there is no strong imbalance, the rest of the classes are about the same

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(data= train, y='Neighborhood', palette='rainbow', x='SalePrice');

This variable strongly affects the target one, for example, **NoRidge** and **Mitchell** have approximately the same class distribution, but **SalePrice** is very different for them

#### Condition1

**Condition1**: Proximity to main road or railroad

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data=train, x='Condition1')
plt.xticks(rotation=45);

There is a strong class imbalance, **Norm** is more common than the rest

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(data= train, y='Condition1', x='SalePrice');

All other classes fall into the **Norm** whisker zone

#### SaleType, HouseStyle

**SaleType**: Type of sale

**HouseStyle**: Style of dwelling

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.countplot(data=train, x='SaleType', palette='Set1', ax=ax[0])
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=45, horizontalalignment='right')
sns.countplot(data=train, x='HouseStyle', palette='Set1', ax=ax[1])
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=45, horizontalalignment='right');

**SaleType** has an imbalance of classes, **WD** prevails

**HouseStyle** has two classes 1-2 Story, more often found above the rest

In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,10))
sns.boxplot(data= train, y='SaleType', x='SalePrice', palette='Set1', ax=ax[0])
sns.boxplot(data= train, y='HouseStyle', x='SalePrice', palette='Set1', ax=ax[1]);

The **WD** and **New** classes in the **SaleType** class have different effects on the target variable, although the **New** class is less common in the data, but it definitely has a dependency on **SalePrice**

In **HouseStyle**, the 3 most popular classes describe the **Sale Price** well

Great, after analyzing all the selected variables, it can be said that the **Neighborhood** has the greatest influence on the price of a house.

Now let's look at the rest of the variables, but let's make it faster and more compact.

As I said above, we will divide the remaining variables into 5 parts and look at them

### Other categorical columns

In [ ]:
train_object = train_object.drop(
    ['Neighborhood', 'Exterior1st', 'Exterior2nd', 'Condition1', 'SaleType', 'HouseStyle'],
    axis = 1)

In [ ]:
object_part_1 = train_object.columns.to_list()[0:5]
object_part_2 = train_object.columns.to_list()[5:9]
object_part_3 = train_object.columns.to_list()[9:13]
object_part_4 = train_object.columns.to_list()[13:17]
object_part_5 = train_object.columns.to_list()[17:]

#### part_1

In [ ]:
fig, ax =plt.subplots(2,3, figsize=(15,10))
sns.countplot(data=train, x=object_part_1[0], palette='Set2', ax=ax[0,0])
sns.countplot(data=train, x=object_part_1[1], palette='Set2', ax=ax[0,1])
sns.countplot(data=train, x=object_part_1[2], palette='Set2', ax=ax[0,2])
sns.countplot(data=train, x=object_part_1[3], palette='Set2', ax=ax[1,0])
sns.countplot(data=train, x=object_part_1[4], palette='Set2', ax=ax[1,1]);

Strong imbalance of all classes, not counting **LotShape**

In [ ]:
fig, ax =plt.subplots(2,3, figsize=(15,10))
sns.boxplot(data= train, y=object_part_1[0], x='SalePrice', palette='Set2', ax=ax[0,0])
sns.boxplot(data= train, y=object_part_1[1], x='SalePrice', palette='Set2', ax=ax[0,1])
sns.boxplot(data= train, y=object_part_1[2], x='SalePrice', palette='Set2', ax=ax[0,2])
sns.boxplot(data= train, y=object_part_1[3], x='SalePrice', palette='Set2', ax=ax[1,0])
sns.boxplot(data= train, y=object_part_1[4], x='SalePrice', palette='Set2', ax=ax[1,1]);

There is a clear influence on the price in the **MSZoning** variable, but it should be borne in mind that the **RL** class predominates

There is also a dependency in the **Lot Shape** variable

#### part_2

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.countplot(data=train, x=object_part_2[0], palette='Set3', ax=ax[0,0])
sns.countplot(data=train, x=object_part_2[1], palette='Set3', ax=ax[0,1])
sns.countplot(data=train, x=object_part_2[2], palette='Set3', ax=ax[1,0])
sns.countplot(data=train, x=object_part_2[3], palette='Set3', ax=ax[1,1]);

The situation is much better, the variables have an imbalance, but they have at least 2 main classes

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.boxplot(data= train, y=object_part_2[0], x='SalePrice', palette='Set3', ax=ax[0,0])
sns.boxplot(data= train, y=object_part_2[1], x='SalePrice', palette='Set3', ax=ax[0,1])
sns.boxplot(data= train, y=object_part_2[2], x='SalePrice', palette='Set3', ax=ax[1,0])
sns.boxplot(data= train, y=object_part_2[3], x='SalePrice', palette='Set3', ax=ax[1,1]);

We can see the effect on the target variable depending on the classes, this is well expressed in **ExterQual** and **Foundation**

#### part_3

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.countplot(data=train, x=object_part_3[0], palette='pastel', ax=ax[0,0])
sns.countplot(data=train, x=object_part_3[1], palette='pastel', ax=ax[0,1])
sns.countplot(data=train, x=object_part_3[2], palette='pastel', ax=ax[1,0])
sns.countplot(data=train, x=object_part_3[3], palette='pastel', ax=ax[1,1]);

Good class distribution in **BsmtFinType1** 

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.boxplot(data= train, y=object_part_3[0], x='SalePrice', palette='pastel', ax=ax[0,0])
sns.boxplot(data= train, y=object_part_3[1], x='SalePrice', palette='pastel', ax=ax[0,1])
sns.boxplot(data= train, y=object_part_3[2], x='SalePrice', palette='pastel', ax=ax[1,0])
sns.boxplot(data= train, y=object_part_3[3], x='SalePrice', palette='pastel', ax=ax[1,1]);

**BsmtQual** has a strong effect on the target variable

You can also see that with a light class balance in **BsmtFinType1**, you can see how the **GLQ** class affects the target variable

#### part_4

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.countplot(data=train, x=object_part_4[0], palette='rainbow', ax=ax[0,0])
sns.countplot(data=train, x=object_part_4[1], palette='rainbow', ax=ax[0,1])
sns.countplot(data=train, x=object_part_4[2], palette='rainbow', ax=ax[1,0])
sns.countplot(data=train, x=object_part_4[3], palette='rainbow', ax=ax[1,1]);

Good distribution in all variables except **BsmtFinType2**

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.boxplot(data= train, y=object_part_4[0], x='SalePrice', palette='rainbow', ax=ax[0,0])
sns.boxplot(data= train, y=object_part_4[1], x='SalePrice', palette='rainbow', ax=ax[0,1])
sns.boxplot(data= train, y=object_part_4[2], x='SalePrice', palette='rainbow', ax=ax[1,0])
sns.boxplot(data= train, y=object_part_4[3], x='SalePrice', palette='rainbow', ax=ax[1,1]);

All variables except **BsmtFinType2** have an effect on **SalePrice**

#### part_5

In [ ]:
fig, ax =plt.subplots(3,2, figsize=(15,10))
sns.countplot(data=train, x=object_part_5[0], palette='coolwarm', ax=ax[0,0])
sns.countplot(data=train, x=object_part_5[1], palette='coolwarm', ax=ax[0,1])
sns.countplot(data=train, x=object_part_5[2], palette='coolwarm', ax=ax[1,0])
sns.countplot(data=train, x=object_part_5[3], palette='coolwarm', ax=ax[1,1]);
sns.countplot(data=train, x=object_part_5[4], palette='coolwarm', ax=ax[2,0]);

Good class distribution at **GarageFinish**

In [ ]:
fig, ax =plt.subplots(3,2, figsize=(15,10))
sns.boxplot(data= train, y=object_part_5[0], x='SalePrice', palette='coolwarm', ax=ax[0,0])
sns.boxplot(data= train, y=object_part_5[1], x='SalePrice', palette='coolwarm', ax=ax[0,1])
sns.boxplot(data= train, y=object_part_5[2], x='SalePrice', palette='coolwarm', ax=ax[1,0])
sns.boxplot(data= train, y=object_part_5[3], x='SalePrice', palette='coolwarm', ax=ax[1,1]);
sns.boxplot(data= train, y=object_part_5[4], x='SalePrice', palette='coolwarm', ax=ax[2,0]);

**GarageFinish** has a good dependency

-----------------

After analyzing all the categorical variables, it can be concluded that many of them have a strong influence on the target variable.

Many variables after cleaning still have a strong imbalance, and some of them have an almost linear relationship with the target variable


Now you need to deal with the missing values in the data

# Preprocessing

# Filling in Missing Values

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(train.isnull());

In [ ]:
train.isnull().sum().head(60)

There is a relationship between missings and variables

**Bsmt** have 37-38 missing values

**Garage**  have 80 missing values

**MasVnrType** and **MasVnrArea** each have 8 missing values

Let's take a look at them individually and decide what to do with them, after that we'll deal with **LotFrontage**

We will carry out all operations on train and test, if there are missings in test, we will analyze them separately

## Train

### Bsmt missing values

**BsmtQual**: Height of the basement

**BsmtCond**: General condition of the basement

**BsmtExposure**: Walkout or garden level basement walls

**BsmtFinType1**: Quality of basement finished area

**BsmtFinType2**: Quality of second finished area (if present)

In [ ]:
Bsmt = train.filter(like="Bsmt")

print(
     'BsmtQual:', Bsmt.BsmtQual.unique(),
     '\nBsmtCond:', Bsmt.BsmtCond.unique(),
     '\nBsmtExposure:', Bsmt.BsmtExposure.unique(),
     '\nBsmtFinType1:', Bsmt.BsmtFinType1.unique(),
     '\nBsmtFinType2:', Bsmt.BsmtFinType2.unique()
     )

Looks like these 37-38 houses just don't have a basement

We have already introduced a variable that is responsible for the presence of a basement in the house, let's look at houses that do not have a basement

In [ ]:
Bsmt[Bsmt['CheckBsmt']==0].isnull().sum()

Great, these are exactly the values we were looking for, let's convert **NaN to No**, which will indicate that this parameter is missing

In [ ]:
train['BsmtQual'].fillna('No', inplace=True)
train['BsmtCond'].fillna('No', inplace=True)
train['BsmtExposure'].fillna('No', inplace=True)
train['BsmtFinType1'].fillna('No', inplace=True)
train['BsmtFinType2'].fillna('No', inplace=True)

test['BsmtQual'].fillna('No', inplace=True)
test['BsmtCond'].fillna('No', inplace=True)
test['BsmtExposure'].fillna('No', inplace=True)
test['BsmtFinType1'].fillna('No', inplace=True)
test['BsmtFinType2'].fillna('No', inplace=True)

Bsmt = train.filter(like="Bsmt")
Bsmt.isnull().sum()

Great, now let's look at the **Garage**, but I'm sure the missing values are due to the fact that the garage is missing

### Garage missing values 

**GarageType**: Garage location

**GarageYrBlt**: Year garage was built

**GarageFinish**: Interior finish of the garage

**GarageQual**: Garage quality


In [ ]:
Garage = train.filter(like="Garage")

In [ ]:
print(
     'GarageType:', Garage.GarageType.unique(),
     '\nGarageFinish:', Garage.GarageFinish.unique(),
     '\nGarageQual:', Garage.GarageQual.unique(),
     '\nGarageYrBlt:', Garage.GarageYrBlt.unique()
     )

We introduced a variable for the presence of a garage in the house, which will make it clear that there is no garage


In [ ]:
Garage[Garage['CheckGarage']==0].isnull().sum()

Indeed, these are the same values responsible for the absence of a garage in the house, let's convert them to **No**

But we also have **GarageYrBlt**, which is responsible for the year of construction of the garage. We do not want to write **No** for this variable, because we will automatically make it an object type, but we do not want to fill **0**, because this can worsen the understanding of this feature, so there is nothing left but to remove this feature

In [ ]:
train['GarageType'].fillna('No', inplace=True)
train['GarageFinish'].fillna('No', inplace=True)
train['GarageQual'].fillna('No', inplace=True)

test['GarageType'].fillna('No', inplace=True)
test['GarageFinish'].fillna('No', inplace=True)
test['GarageQual'].fillna('No', inplace=True)

train.drop(['GarageYrBlt'], axis=1, inplace=True)
test.drop(['GarageYrBlt'], axis=1, inplace=True)

Garage = train.filter(like="Garage")
Garage.isnull().sum()

### MasVnrType and MasVnrArea  missing values

**MasVnrType**: Masonry veneer type

**MasVnrArea**: Masonry veneer area in square feet

In [ ]:
Mas = train.filter(like="Mas")
Mas[Mas['MasVnrArea']==0].isnull().sum()

The logic here is the same, there is no masonry cladding, there is no type of this masonry, fill in **No** and **0**

In [ ]:
train['MasVnrType'].fillna('No', inplace=True)
train['MasVnrArea'].fillna(0, inplace=True)

test['MasVnrType'].fillna('No', inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)

Great, now let's take a look at the **LotFrontage**

### LotFrontage missing values

LotFrontage: Linear feet of street connected to property

In [ ]:
train['LotFrontage'].unique()

Perhaps **NaN** is also responsible for the absence, let's see if there are 0 values in this variable

In [ ]:
train['LotFrontage'][train['LotFrontage']==0].sum()

There are no data values, so fill in the missings how **0**

In [ ]:
train['LotFrontage'].fillna(0, inplace=True)

test['LotFrontage'].fillna(0, inplace=True)

In [ ]:
train.isnull().sum().any()

## Test

In [ ]:
test.isnull().sum().tail(10)

For categorical variables, we write their unique values, for nominative variables, the number of unique values

In [ ]:
print(
     'MSZoning:', test.MSZoning.unique(),
     '\nExterior1st:', test.Exterior1st.unique(),
     '\nExterior2nd:', test.Exterior2nd.unique(),
     '\nTotalBsmtSF count:', test.TotalBsmtSF.nunique(),
     '\nBsmtFinSF1 count:', test.BsmtFinSF1.nunique(),
     '\nBsmtFinSF2 count:', test.BsmtFinSF2.nunique(),
     '\nBsmtUnfSF count:', test.BsmtUnfSF.nunique(),
     '\nBsmtFullBath:', test.BsmtFullBath.unique(),
     '\nBsmtHalfBath:', test.BsmtHalfBath.unique(),
     '\nKitchenQual:', test.KitchenQual.unique(),
     '\nGarageArea count:', test.GarageArea.nunique(),
     '\nSaleType:', test.SaleType.unique(),
     '\nTotalSF count:', test.TotalSF.nunique(),
     )

As we saw earlier, all this data cannot but have its own values, most likely these are really missings in the data.

There are not too many missings, so it makes no sense to apply exact methods to fill in missing values

Let's fill them with the first non-missing value that occurs before it.

In [ ]:
test['MSZoning'].fillna(method='ffill', inplace=True)
test['Exterior1st'].fillna(method='ffill', inplace=True)
test['Exterior2nd'].fillna(method='ffill', inplace=True)
test['TotalBsmtSF'].fillna(method='ffill', inplace=True)
test['BsmtFinSF1'].fillna(method='ffill', inplace=True)
test['BsmtFinSF2'].fillna(method='ffill', inplace=True)
test['BsmtUnfSF'].fillna(method='ffill', inplace=True)
test['BsmtFullBath'].fillna(method='ffill', inplace=True)
test['BsmtHalfBath'].fillna(method='ffill', inplace=True)
test['KitchenQual'].fillna(method='ffill', inplace=True)
test['GarageArea'].fillna(method='ffill', inplace=True)
test['SaleType'].fillna(method='ffill', inplace=True)
test['TotalSF'].fillna(method='ffill', inplace=True)

In [ ]:
test.isnull().sum().any()

Great, all missing values have been restored


## Encoding

To begin with, let's temporarily combine our datasets so that our transformations are done the same for **test** and **train**, this could have been done at the stage of filling in the missing values, but for some reason I did not think about it before

Next, we divide our variables into **categorical** and **nominative** feature.

After all the transformations, we use **join** to combine them and again **split** the dataset into **train** and **test**

We will also straightaway create **y_train** as a target variable, before that we **reset the indexes**, since we removed outliers in our train data

In [ ]:
train.reset_index(inplace=True)
y_train = train['SalePrice']
train.drop(['SalePrice'],axis=1, inplace=True)

In [ ]:
train_size = train.shape[0]
test_size = test.shape[0]
df = pd.concat((train, test)).reset_index(drop=True)
df.shape

In [ ]:
df_object = df.select_dtypes(include=['object'])
df_num = df.select_dtypes(exclude=['object'])

In [ ]:
df_object.shape, df_num.shape

### Categorical features

The need to combine data was exactly because of **get_dummies**, if we used **get_dummies** on different data, we could get a different number of columns in train and test, as some classes in features might not be in test, but they were in train, because of this we would have errors in the future

In [ ]:
df_object = pd.get_dummies(df_object)

In [ ]:
df_object.shape

### Nominative features

To scale the nominative features, we will use **StandardScaler**, since in the future we will use **PCA**, which requires scaling with a center at 0 and a variance of 1

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scal = StandardScaler()
df_num_scal = scal.fit_transform(df_num)
df_num_scal = pd.DataFrame(df_num_scal, columns=df_num.columns, index= df_num.index)

### Data aggregation

Join our processed data with **join**

Since the default is the inner setting, we will not change anything

In [ ]:
X_df = df_num_scal.join(df_object)
X_df.shape

In [ ]:
X_df.drop(['index'],axis=1, inplace=True)

Remove the index column that appeared after concatenating

Now let's return our data to the form test and train, which was before the concatenating

In [ ]:
X_train = X_df[:train_size]
X_test = X_df[train_size:]

In [ ]:
X_train.shape, X_test.shape

# Principal component analysis

Use **PCA** to eliminate multicollinearity between features and get new **X_train** and **X_test** consisting of principal components

It should be understood that the main components do not mean the signs that we analyzed above.

The main component is a new features, a mixture of the original ones.

These combinations are performed in such a way that the principal components are not correlated, and most of the information in the original features is placed in the first components.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(random_state=17)
pca.fit_transform(X_train)
pca.transform(X_test);

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.xlim(0, 224)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(66, c='b')
plt.axhline(0.95, c='r')
plt.show();

Great, we see that for 95% we need 66 main components, but we will keep all the components, since in this case we are not trying to reduce the dimension.

Our goal is to eliminate **possible multicollinearity**

In [ ]:
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Machine Learning

In this work, I will practice ensembles of models without affecting boosting, for this method, I will make another notebook

This project will use models such as:

LinearRegression

Lasso

Ridge

elasticnet

SVR

Kernel Ridge

SGDRegressor

RandomForestRegressor

First, import all the necessary libraries


In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, KFold

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import learning_curve

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor

--------------------

We will at once create kfold, as well as create the necessary functions for quickly displaying the necessary information

In [ ]:
kf = KFold(n_splits=5 , shuffle=True, random_state=17)

In [ ]:
def print_metrics(estimator):  
    mae = -cross_val_score(estimator, X_train, y_train, cv = kf, scoring="neg_mean_absolute_error", n_jobs=-1)
    mse = -cross_val_score(estimator, X_train, y_train, cv = kf, scoring="neg_mean_squared_error", n_jobs=-1)
    rmse = np.sqrt(-cross_val_score(estimator, X_train, y_train, cv = kf, scoring="neg_mean_squared_error", n_jobs=-1))
    r2 = cross_val_score(estimator, X_train, y_train, cv = kf, scoring="r2", error_score='raise', n_jobs=-1)
    print(estimator,'\n--------------------------------')
    print('MAE:', np.mean(mae))
    print('MSE:', np.mean(mse))
    print('RMSE:', np.mean(rmse))
    print('R2:', np.mean(r2))
    print('--------------------------------\n')

In [ ]:
#def predict_metrics(y_train, pred):  
#    mae = mean_absolute_error(y_train, pred)
#    mse = mean_squared_error(y_train, pred)
#    rmse = np.sqrt(mean_squared_error(y_train, pred))
#    r2 = r2_score(y_train, pred)
#    print('MAE:', mae)
#    print('MSE:', mse)
#    print('RMSE:', rmse)
#    print('R2:', r2)
#    print('')

In [ ]:
def check_rmse(estimator):  
    rmse = np.sqrt(-cross_val_score(estimator, X_train, y_train, cv = kf, scoring="neg_mean_squared_error", n_jobs=-1))
    return np.mean(rmse)

In [ ]:
def predict_rmse(y_train, pred):
    rmse = np.sqrt(mean_squared_error(y_train, pred))
    return np.mean(rmse)

Great, now we will train the basic models and look at the results, then we will find the optimal parameters for them

## Base models

In [ ]:
%%time
lr = LinearRegression().fit(X_train, y_train)
lasso = Lasso(random_state=17).fit(X_train, y_train)
ridge = Ridge(random_state=17).fit(X_train, y_train)
elastic = ElasticNet(random_state=17).fit(X_train, y_train)
svr = SVR().fit(X_train, y_train)
kr = KernelRidge().fit(X_train, y_train)
sgd = SGDRegressor(random_state=17).fit(X_train, y_train)
rs = RandomForestRegressor(random_state=17, n_jobs=-1).fit(X_train, y_train)

In [ ]:
print_metrics(lr), 
print_metrics(lasso), 
print_metrics(ridge),
print_metrics(elastic),
print_metrics(svr),
print_metrics(kr),
print_metrics(sgd), 
print_metrics(rs)


**Linear Regression** is badly trained, large **MAE** and **MSE**, as well as negative **R2** indicates that the model performs worse than random distribution

**Kernel ridge** is very badly trained, perhaps this is due to the fact that the standard **Kernel ridge** uses **kernel = linear**, which is not suitable for the data

Standard **Lasso** and **ElasticNet** output bad results, this may mean that **L1** regularization or standard **alpha for L1** is not suitable for this goal. Since standart **ElasticNet** uses **0.5 L1** when training the model, this could affect the result

**Ridge**, **SGDRegressor**, **SVR** and **Random Forest** showed good results


--------------------

Explicitly specified hyperparameters will improve the situation, let's find the best parameters for each model 

We will no longer use the standard Linear Regression in this work


## Hyperparameter tuning

In [ ]:
def grid_search (estimator, param):
    gscv = GridSearchCV(estimator, param, cv=kf, n_jobs=-1, scoring='neg_mean_squared_error')
    gscv.fit(X_train, y_train)
    return gscv.best_estimator_

def random_search (estimator, param):
    rscv = RandomizedSearchCV(estimator, param, cv=kf, n_jobs=-1, scoring='neg_mean_squared_error')
    rscv.fit(X_train, y_train)
    return rscv.best_estimator_

In [ ]:
lasso_params = {'alpha':[.0005, .001, .0015, .01, .015, .1, 1]}
ridge_params = {'alpha':[.01, .015, .1, 1, 10, 100]}
sgd_params = {'alpha':[.001, .01, .1, 1], 'eta0':[.001, .003, .01, .03, .1, .3, 1]}
elastic_params ={'alpha':[.0005, .001, .0015, .01, .015, .1, 1], 'l1_ratio':[.1, .3, .5, .7, .9]}
svr_params = {'C': [.001, .015, .01, .1], 'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
kr_params = {'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
rs_params = {'n_estimators': range(100, 551, 50)}

gscv_param = [lasso_params, ridge_params, elastic_params, sgd_params, kr_params]
rscv_param = [svr_params, rs_params]

I decided not to write a huge number of different parameters, but limited myself to the most important ones for each model.

Also, for faster parameter finding, for long-learning models such as **RandomForest** or **SVR**, I will use **RandomSearch**


In [ ]:
%%time
best_est = []
n=0
for model in [lasso, ridge, elastic, sgd, kr]:
    best_est.append(grid_search(model, gscv_param[n]))
    n+=1
n=0
for model in [svr, rs]:
    best_est.append(random_search(model, rscv_param[n]))
    n+=1

In [ ]:
best_est

In [ ]:
for model in best_est:
    print_metrics(model)

Great! After hyper-tuning, all our models performed much better, with the exception of **Random Forest**: hyper-tuning improved the results, but not significantly compared to the base model

For further work speed up, I will not use Random Forest, because training this model takes a lot of time, and the result is worse than other algorithms one

Let's reassign the model data as the main model and save the results

As the data on Kaggle imply checking for **RMSE**, in order to reduce the training time and calculate metrics, in the future I will only consider **RMSE**

In [ ]:
best_est

In [ ]:
best_est = best_est[:6]

lasso = best_est[0]
ridge = best_est[1]
elastic = best_est[2]
sgd = best_est[3]
kr = best_est[4]
svr = best_est[5]

lasso_rmse = predict_rmse(y_train, lasso.predict(X_train))
ridge_rmse = predict_rmse(y_train, ridge.predict(X_train))
elastic_rmse = predict_rmse(y_train, elastic.predict(X_train))
sgd_rmse = predict_rmse(y_train, sgd.predict(X_train))
kr_rmse = predict_rmse(y_train, kr.predict(X_train))
svr_rmse = predict_rmse(y_train, svr.predict(X_train))

Now let's start trying ensemble methods, ranging from simple ones to more complex ones.

Although we have already used Random Forest. This model belongs to the **bagging method** and implements its algorithm, so I will not try to apply **BaggingRegressor** in this notebook, especially Random Forest shows itself worse than other regression models.

**Bagging** and **boosting** are needed to reduce one of the trade-off metrics: **bias** and **variance**.
**Staking** does not try to minimize any of these errors.
Successfully stacking simply reduces the error, and as a result, its components will also decrease.

In [ ]:
def plot_learning_curve(estimator, X, y):
    
    train_sizes, train_score, test_score = learning_curve(
    estimator =  estimator,
    X = X,
    y = y, 
    train_sizes = np.linspace(0.01, 1.0, 50), 
    cv = kf,
    n_jobs=-1,
    shuffle=True,
    scoring='neg_mean_squared_error',
    random_state=17) 
    
    mean_train = -np.mean(train_score, axis=1)
    mean_test = -np.mean(test_score, axis=1)

    plt.plot(train_sizes, mean_train, '--', color="b",  label="Training score")
    plt.plot(train_sizes, mean_test ,color="g", label="Cross-validation score")

    plt.title('Learning curve')
    plt.xlabel("Size"),
    plt.ylabel("Rmse"),
    plt.legend(loc="best")
    plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plot_learning_curve(svr, X_train, y_train)

In this case, we do not have a large **bias** or **variance**, and reducing these metrics is not the main goal of this notebook

Let me remind you that **boosting** is also not intentionally used in this work, the goal of this notebook is not to get the best result, but to learn how to apply only some **ensemble methods** of machine learning.

# Ensemble methods

Here I will try ensemble algorithms like:

1. **Voting Ensembles**
2. **Weight voting Ensembles**
3. **Stacking**

## Voting Ensembles

This method is more often used in classification problems, but you can also try it for regression.

The method is that we take the arithmetic mean of our predictions

In [ ]:
%%time
voting = VotingRegressor([('lasso',lasso),('ridge',ridge),('elastic', elastic),('svr',svr),('kr',kr),('sgd',sgd)], n_jobs=-1)
voting.fit(X_train, y_train)

In [ ]:
print_metrics(voting)

In [ ]:
voting_rmse = predict_rmse(y_train, voting.predict(X_train))

## Weight voting Ensembles

In this method, we must give our models **weights** that will decide the overall contribution of a particular model to the prediction.

Let's look at the results of our models on the chart and decide what **weights** to assign to each model.

In [ ]:
scores = {}
for model in best_est:
    scores[str(model)[:(str(model).find('('))]] = check_rmse(model)

In [ ]:
plt.figure(figsize=(15,8))
sns.pointplot(y = list(scores.values()), x=list(scores.keys()),markers='X');

In [ ]:
w1, w2, w3, w4, w5, w6 = 0.16, 0.18, 0.2, 0.03, 0.35, 0.08

In [ ]:
weight_voting = VotingRegressor([('lasso',lasso),('ridge',ridge),('elastic', elastic),('sgd',sgd),('kr',kr),('svr',svr)],
                                weights=[w1, w2, w3, w4, w5, w6],
                                n_jobs=-1)
weight_voting.fit(X_train, y_train)

In [ ]:
print_metrics(weight_voting)

In [ ]:
weight_voting_rmse = predict_rmse(y_train, weight_voting.predict(X_train))

## Stacking 

In this method, we use cross-validation to get the predictions of the **base model**, then these predictions are passed to the **meta-model**, which makes the final prediction

In [ ]:
%%time
stack = StackingRegressor([('lasso',lasso),('ridge',ridge),('svr',svr),('sgd',sgd),('kr',kr),('elastic',elastic)], kr, cv=kf, n_jobs=-1)
stack.fit(X_train, y_train)

In [ ]:
print_metrics(stack)

In [ ]:
stack_rmse = predict_rmse(y_train, stack.predict(X_train))

# Result and submission

In [ ]:
result =[lasso_rmse, ridge_rmse, elastic_rmse, sgd_rmse, kr_rmse, svr_rmse, voting_rmse, weight_voting_rmse, stack_rmse]
model_name =['Lasso','Ridge','Elastic Net','SGDRegressor','Kernel Ridge','SVR','Voting','Weight voting','Stacking']

In [ ]:
result_rmse = pd.DataFrame(result, index=model_name, columns=['RMSE']).sort_values(['RMSE']).T
result_rmse

In [ ]:
plt.figure(figsize=(15,10))
sns.pointplot(y = result, x=model_name,markers='X');

In our notebook, the best model was **Stacking Model**, but I want to present the average of the best models, this should improve accuracy even more.

In [ ]:
final_model = VotingRegressor([('ridge',ridge),('elastic', elastic),('kr',kr),('voting', voting),('weight_voting',weight_voting),('stack',stack)],
                                weights=[0.1, 0.1, 0.2, 0.1, 0.15, 0.35],
                                n_jobs=-1).fit(X_train, y_train)

In [ ]:
predict = np.exp(final_model.predict(X_test))

In [ ]:
submission = pd.DataFrame({'Id':y_test.Id, 'SalePrice': predict})

In [ ]:
submission.to_csv('submission_house_price_final.csv', index=False)

In [ ]:
submission.head()

## Thank you for watching it's my project, I will be grateful if you give feedback on my work in comments. I want to improve my skills and if you find mistakes in work, tell me about it, please.